In [1]:
import pandas as pd 
import numpy as np
import os
import re

In [173]:
m_lens = 2 # Which lens is m assigned to
m_param = 5 # Which parameter of that lens is m assigned to
m = np.linspace(0.1, 1.0, 10)
n_lens = 2
n_param = 6
n = np.linspace(0.1, 1.0, 10)
o_lens = 1
o_param = 8
o = np.linspace(0.1, 1.0, 10)

In [182]:
path = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/System 2/'
name = 'SIE'

In [183]:
with open ('/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Simulations/input.py') as f:
    lines = f.readlines()

    # Find and Replace
    for i, line in enumerate(lines):
        if line.strip().startswith('path ='):
            lines[i] = f"path = '{path}/{name}'\n"
        elif line.strip().startswith('constraint_file ='):
            lines[i] = "constraint_file = path + 'pos_point.dat'\n"
    
    # Find all occurrences of 'glafic.set_lens('
    lens_lines = [i for i, line in enumerate(lines) if line.strip().startswith('glafic.set_lens(')]

    # Modify the m, n, o values in the appropriate lens line
    for lens_index, param_index, value in [(m_lens, m_param, m),
                                        (n_lens, n_param, n),
                                        (o_lens, o_param, o)]:
        line_index = lens_lines[lens_index - 1]
        line = lines[line_index]

        # Extract content inside parentheses
        inside = re.search(r'\((.*)\)', line).group(1)

        # Split arguments
        parts = [p.strip() for p in inside.split(',')]

        # The first two arguments are: lens_id, 'model'
        start_index = 2   # parameters start from index 2 (0-based)

        # param_index is 1-based relative to *physical parameters* after the model
        target_index = start_index + (param_index - 1)

        if 0 <= target_index < len(parts):
            parts[target_index] = str(value[0])
        else:
            print(f"⚠️ Warning: param_index {param_index} out of range for line:\n{line}")

        # Rebuild the line
        new_inside = ', '.join(parts)
        lines[line_index] = re.sub(r'\(.*\)', f'({new_inside})', line)


# Write back the modified lines
with open('/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Simulations/input.py', 'w') as f:
    f.writelines(lines)


In [2]:
pow_params = ['$z_{s,fid}$', 'x', 'y', 'e', '$θ_{e}$', '$r_{Ein}$', '$\gamma$ (PWI)']

# SIE
sie_params = ['$\sigma$', 'x', 'y', 'e', '$θ_{e}$', '$r_{core}$', 'NaN']

# NFW
nfw_params = ['M', 'x', 'y', 'e', '$θ_{e}$', 'c or $r_{s}$', 'NaN']

# EIN
ein_params = ['M', 'x', 'y', 'e', '$θ_{e}$', 'c or $r_{s}$', r'$\alpha_{e}$']

# SHEAR 
shear_params = ['$z_{s,fid}$', 'x', 'y', '$\gamma$', '$θ_{\gamma}$', 'NaN', '$\kappa$']

# Sersic
sersic_params = ['$M_{tot}$', 'x', 'y', 'e', '$θ_{e}$', '$r_{e}$', '$n$']

# Cored SIE
cored_sie_params = ['M', 'x', 'y', 'e', '$θ_{e}$', '$r_{core}$', 'NaN']

# Multipoles
mpole_params = ['$z_{s,fid}$', 'x', 'y', '$\epsilon$', '$θ_{m}$', 'm', 'n']

model_list = ['POW', 'SIE', 'ANFW', 'EIN', 'PERT', 'SERS', 'MPOLE']
model_params = {
    'POW': pow_params,
    'SIE': sie_params,
    'ANFW': nfw_params,
    'EIN': ein_params,
    'PERT': shear_params,
    'SERS': sersic_params,
    'MPOLE' : mpole_params
}

In [79]:
obs_point_file = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/obs_point/obs_point_(POS+FLUX).dat' 
input_py_file = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/Simulations/input.py'  # Observation file path

In [ ]:
def rms_extract(model_ver, model_path, constraint):
    global pos_rms, mag_rms, chi2_value
    # Load the data
    with open(model_path + '/' + model_ver + '_optresult' + '.dat', 'r') as file:
        opt_result = file.readlines()

    # Find the last line with 'optimize' in it
    last_optimize_index = None
    for idx in range(len(opt_result) - 1, -1, -1):
        if 'optimize' in opt_result[idx]:
            last_optimize_index = idx
            last_optimize_line = opt_result[idx]
            break
    if last_optimize_index is None:
        raise ValueError("No line with 'optimize' found in the file.")

    # Extract everything after the last 'optimize' line
    opt_result = opt_result[last_optimize_index + 1:]

    # Count the number of lines that start with 'lens'
    lens_count = sum(1 for line in opt_result if line.startswith('lens'))

    # Initialize a dictionary to hold the lens parameters
    lens_params_dict = {}

    # Extract the lens parameters
    lens_params = []
    for line in opt_result:
        if line.startswith('lens'):
            parts = re.split(r'\s+', line.strip())
            lens_name = parts[1]
            params = [float(x) for x in parts[2:]]

            # Store the parameters in the dictionary
            lens_params_dict[lens_name] = params
            lens_params.append((lens_name, params))

    # Remove the first lens parameter
    if lens_params:
        for i in range(len(lens_params)):
            lens_name, params = lens_params[i]
            lens_params_dict[lens_name] = params[1:]

    # Extract the source parameters
    source_params = []
    for line in opt_result:
        if line.startswith('point'):
            parts = re.split(r'\s+', line.strip())
            params = [float(x) for x in parts[1:]]
            source_params.append(params)
    
    # Extract the chi2 
    chi2_line = next((line for line in opt_result if 'chi^2' in line), None)
    if chi2_line is None:
        raise ValueError("No line with 'chi2' found in the file.")

    chi2_value = float(chi2_line.split('=')[-1].strip().split()[0])
    # print(f"✅ Extracted chi2 value: {chi2_value}")

    # Number of len profiles
    num_lens_profiles = len(lens_params_dict)

    # Use generic column names: param1, param2, ...
    df = pd.DataFrame()
    rows = []
    max_param_len = 0

    for lens_name, params in lens_params_dict.items():
        row = {'Lens Name': lens_name}
        for i, val in enumerate(params):
            row[f'param{i+1}'] = val
        rows.append(row)
        if len(params) > max_param_len:
            max_param_len = len(params)

    columns = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len)]
    df = pd.DataFrame(rows, columns=columns)
    
    # Load the input parameters from the Python file
    with open(input_py_file, 'r') as file:
        py = file.readlines()

    # Extracting the input parameters from the Python file
    set_lens_lines = [line for line in py if line.startswith('glafic.set_lens(')]
    if not set_lens_lines:
        raise ValueError("No lines starting with 'glafic.set_lens(' found in the file.")

    set_lens_params = []
    for line in set_lens_lines:
        match = re.search(r'set_lens\((.*?)\)', line)
        if match:
            params_str = match.group(1)
            params = [param.strip() for param in params_str.split(',')]
            set_lens_params.append(params)
        else:
            raise ValueError(f"No valid parameters found in line: {line.strip()}")

    # Store the parameters in a dictionary
    set_lens_dict = {}
    for params in set_lens_params:
        if len(params) < 3:
            raise ValueError(f"Not enough parameters found in line: {params}")
        lens_name = params[1].strip("'\"")  # Remove quotes from lens name
        lens_params = [float(x) for x in params[2:]]  # Skip index and lens name
        set_lens_dict[lens_name] = lens_params

    # Remove the first lens parameter
    if set_lens_dict:
        for lens_name, params in set_lens_dict.items():
            set_lens_dict[lens_name] = params[1:]  # Remove the first parameter (index)

    # Use generic column names: param1, param2, ...
    df_input = pd.DataFrame()
    rows_input = []
    max_param_len_input = 0
    for lens_name, params in set_lens_dict.items():
        row = {'Lens Name': lens_name}
        for i, val in enumerate(params):
            row[f'param{i+1}'] = val
        rows_input.append(row)
        if len(params) > max_param_len_input:
            max_param_len_input = len(params)
    columns_input = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len_input)]
    df_input = pd.DataFrame(rows_input, columns=columns_input)
    
    # Extract input flags from the Python file
    set_flag_lines = [line for line in py if line.startswith('glafic.setopt_lens(')]
    if not set_flag_lines:
        raise ValueError("No lines starting with 'glafic.setopt_lens(' found in the file.")
    set_flag_params = []
    for line in set_flag_lines:
        match = re.search(r'setopt_lens\((.*?)\)', line)
        if match:
            params_str = match.group(1)
            params = [param.strip() for param in params_str.split(',')]
            set_flag_params.append(params)
        else:
            raise ValueError(f"No valid parameters found in line: {line.strip()}")
    
    # Store the parameters in a dictionary
    set_flag_dict = {}
    for params in set_flag_params:
        if len(params) < 2:
            raise ValueError(f"Not enough parameters found in line: {params}")
        # The lens name is not present in setopt_lens, so use the lens index to map to set_lens_dict
        lens_index = params[0].strip("'\"")
        # Find the lens name corresponding to this index from set_lens_params
        lens_name = None
        for lens_params in set_lens_params:
            if lens_params[0].strip("'\"") == lens_index:
                lens_name = lens_params[1].strip("'\"")
                break
        if lens_name is None:
            raise ValueError(f"Lens name for index {lens_index} not found in set_lens_params")
        flag = ','.join(params[1:])  # Join all flag values as a string
        set_flag_dict[lens_name] = flag
   
    # Remove the first flag parameter
    if set_flag_dict:
        for lens_name, flag in set_flag_dict.items():
            flag_parts = flag.split(',')
            set_flag_dict[lens_name] = ','.join(flag_parts[1:])  # Remove the first flag parameter
    
    # Dynamically create columns: 'Lens Name', 'flag1', 'flag2', ..., based on the maximum number of flags
    df_flag = pd.DataFrame()
    rows_flag = []
    max_flag_len = 0
    
    # First, determine the maximum number of flags
    for flag in set_flag_dict.values():
        flag_parts = flag.split(',')
        if len(flag_parts) > max_flag_len:
            max_flag_len = len(flag_parts)
    for lens_name, flag in set_flag_dict.items():
        flag_parts = flag.split(',')
        row = {'Lens Name': lens_name}
        for i, val in enumerate(flag_parts):
            row[f'flag{i+1}'] = val
        rows_flag.append(row)
    columns_flag = ['Lens Name'] + [f'flag{i+1}' for i in range(max_flag_len)]  
    df_flag = pd.DataFrame(rows_flag, columns=columns_flag)
    
    # Combine all dataframes into a list of dataframes for each lens
    dfs = []
    
    for i in range(num_lens_profiles):
        lens_name = df['Lens Name'][i]
        
        # Find the model type (case-insensitive match)
        model_type = None
        for m in model_list:
            if m.lower() == lens_name.lower():
                model_type = m
                break
        if model_type is None:
            continue

        symbols = model_params[model_type][:7]
        # Row 2: input
        row_input = pd.DataFrame([df_input.iloc[i, 1:8].values], columns=symbols)
        # Row 3: output
        row_output = pd.DataFrame([df.iloc[i, 1:8].values], columns=symbols)
        # Row 4: flags
        row_flags = pd.DataFrame([df_flag.iloc[i, 1:8].values], columns=symbols)

        # Stack vertically, add a label column for row type
        lens_df = pd.concat([
            row_input.assign(Type='Input'),
            row_output.assign(Type='Output'),
            row_flags.assign(Type='Flag')
        ], ignore_index=True)
        lens_df.insert(0, 'Lens Name', lens_name)
        
        # Move 'Type' to the second column
        cols = lens_df.columns.tolist()
        cols.insert(1, cols.pop(cols.index('Type')))
        lens_df = lens_df[cols]
        dfs.append(lens_df)
    
    # Anomaly Calculation
    columnn_names = ['x', 'y', 'mag', 'pos_err', 'mag_err', '1', '2', '3']
    obs_point = pd.read_csv(obs_point_file, delim_whitespace=True, header=None, skiprows=1, names=columnn_names)
    out_point = pd.read_csv(model_path + '/' + model_ver + '_point.dat', delim_whitespace=True, header=None, skiprows=1, names=columnn_names)
    out_point.drop(columns=['mag_err', '1', '2', '3'], inplace=True)

    # Drop rows in obs_point where the corresponding out_point['mag'] < 1
    mask = abs(out_point['mag']) >= 1
    out_point = out_point[mask[:len(out_point)]].reset_index(drop=True)
    out_point['x_diff'] = abs(out_point['x'] - obs_point['x'])
    out_point['y_diff'] = abs(out_point['y'] - obs_point['y'])
    out_point['mag_diff'] = abs(abs(out_point['mag']) - abs(obs_point['mag']))
    out_point['pos_sq'] = np.sqrt((out_point['x_diff']**2 + out_point['y_diff']**2).astype(float))  # Plotted on graph

    # RMS
    pos_rms = np.average(out_point['pos_sq'])

    mag_rms = np.average(np.sqrt((out_point['mag_diff']**2).astype(float)))

    return pos_rms, mag_rms, dfs, chi2_value

In [30]:
model_name = 'SIE_SHEAR'

In [175]:
num_lenses = model_name.count('_') + 1
macro_model_params = model_name.strip().split('_')[0]
macro_columns = model_params[macro_model_params]
print(macro_columns)

for i in range(1, num_lenses):
    micro_model_params = model_name.strip().split('_')[i]
    if micro_model_params == 'SHEAR':
        micro_model_params = 'PERT'
    micro_columns = model_params[micro_model_params]
    micro_columns = [f'{col}_{i}' for col in micro_columns]

if num_lenses == 1:
    df = pd.DataFrame(columns=['m', 'n', 'o', 'num_images', 'pos_rms', 'mag_rms', 'chi2'] + macro_columns)
else:
    df = pd.DataFrame(columns=['m', 'n', 'o', 'num_images', 'pos_rms', 'mag_rms', 'chi2'] + macro_columns + micro_columns)

model_ver = model_name

if 'POS+FLUX' in model_ver:
    constraint = 'pos_flux'
elif 'POS' in model_ver:
    constraint = 'pos'

constraint = 'pos'
pos_rms, mag_rms, dfs, chi2 = rms_extract(model_name, '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/SIE+SHEAR/', constraint)
if pos_rms == -1: raise IOError("rms_extract failed.")
point_file_path = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/obs_point/obs_point_(POS+FLUX).dat'; num_images = 0
if os.path.exists(point_file_path):
    with open(point_file_path, 'r') as f: num_images = sum(1 for line in f if line.strip()) - 1


# Result dictionary with 'o_param'
# Ensure we're only using the first 7 symbols per model (rms_extract uses [:7] when building dfs)
macro_cols = list(dict.fromkeys(macro_columns[:7])) if isinstance(macro_columns, list) else [macro_columns]
micro_cols = list(dict.fromkeys(micro_columns[:7])) if isinstance(micro_columns, list) else [micro_columns]

def _safe_get(dfs_idx, col):
    if len(dfs) > dfs_idx and col in dfs[dfs_idx].columns:
        return dfs[dfs_idx][col].iloc[0]
    return 0

result_dict = {
    'm': 1,
    'n': 1,
    'o': 1,
    'num_images': num_images,
    'pos_rms': pos_rms,
    'mag_rms': mag_rms,
    'chi2': chi2
}

# Add macro and micro parameters safely
for col in macro_cols:
    result_dict[col] = _safe_get(0, col)

if num_lenses > 1:
    for col in micro_cols:
        result_dict[col] = _safe_get(1, col)

['$\\sigma$', 'x', 'y', 'e', '$θ_{e}$', '$r_{core}$', 'NaN', '$z_{s,fid}$', 'x', 'y', '$\\gamma$', '$θ_{\\gamma}$', 'NaN', '$\\kappa$']


In [176]:
result_dict

{'m': 1,
 'n': 1,
 'o': 1,
 'num_images': 4,
 'pos_rms': 3.101229586803652e-05,
 'mag_rms': 154.45695,
 'chi2': 2.242436e-06,
 '$\\sigma$': 156.3051,
 'x': 0.0,
 'y': 0.0,
 'e': 0.2168966,
 '$θ_{e}$': -1.398259,
 '$r_{core}$': 0.0,
 'NaN': 0.0,
 '$z_{s,fid}$_1': 0,
 'x_1': 0,
 'y_1': 0,
 '$\\gamma$_1': 0,
 '$θ_{\\gamma}$_1': 0,
 'NaN_1': 0,
 '$\\kappa$_1': 0}

In [187]:
pos_rms, mag_rms, dfs, chi2 = rms_extract(model_name, '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/SIE+SHEAR/', constraint)

[[1.0, 20.86899, 20.69488]]


In [7]:
model_name = 'NFW'
model_output_dir = '/Volumes/T7 Shield/Simulations/Output/'
temp_input_py_file = '/Volumes/T7 Shield/Simulations/Input/input.py'
m_val = 0.5
n_val = 0.3
o_val = 0.7
obs_point_file = '/Users/ainsleylewis/Documents/Astronomy/IllustrisTNG Lens Modelling/obs_point/obs_point_(POS+FLUX).dat' 


In [5]:
def rms_extract(model_ver, model_path, temp_input_py_file):
    global pos_rms, mag_rms, chi2_value
    
    opt_result_file = os.path.join(model_path, f'{model_ver}_optresult.dat')
    
    with open(opt_result_file, 'r') as file:
        opt_result = file.readlines()

    last_optimize_index = None
    for idx in range(len(opt_result) - 1, -1, -1):
        if 'optimize' in opt_result[idx]:
            last_optimize_index = idx
            break
    if last_optimize_index is None:
        raise ValueError("No line with 'optimize' found in the file.")

    opt_result = opt_result[last_optimize_index + 1:]

    lens_params_dict = {}
    lens_params = []
    for line in opt_result:
        if line.startswith('lens'):
            parts = re.split(r'\s+', line.strip())
            lens_name = parts[1]
            params = [float(x) for x in parts[2:]]
            lens_params_dict[lens_name] = params
            lens_params.append((lens_name, params))
    if lens_params:
        for i in range(len(lens_params)):
            lens_name, params = lens_params[i]
            lens_params_dict[lens_name] = params[1:]

    source_params = []
    for line in opt_result:
        if line.startswith('point'):
            parts = re.split(r'\s+', line.strip())
            params = [float(x) for x in parts[1:]]
            source_params.append(params)
    
    chi2_line = next((line for line in opt_result if 'chi^2' in line), None)
    if chi2_line is None:
        raise ValueError("No line with 'chi2' found in the file.")
    chi2_value = float(chi2_line.split('=')[-1].strip().split()[0])
    num_lens_profiles = len(lens_params_dict)

    df = pd.DataFrame()
    rows = []
    max_param_len = 0
    for lens_name, params in lens_params_dict.items():
        row = {'Lens Name': lens_name}
        for i, val in enumerate(params): row[f'param{i+1}'] = val
        rows.append(row)
        if len(params) > max_param_len: max_param_len = len(params)
    columns = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len)]
    df = pd.DataFrame(rows, columns=columns)

    with open(temp_input_py_file, 'r') as file:
        py = file.readlines()

    set_lens_lines = [line for line in py if line.startswith('glafic.set_lens(')]
    if not set_lens_lines: raise ValueError("No lines starting with 'glafic.set_lens(' found in the file.")
    set_lens_params = []
    for line in set_lens_lines:
        match = re.search(r'set_lens\((.*?)\)', line)
        if match:
            params_str = match.group(1)
            params = [param.strip() for param in params_str.split(',')]
            set_lens_params.append(params)
        else: raise ValueError(f"No valid parameters found in line: {line.strip()}")

    set_lens_dict = {}
    for params in set_lens_params:
        if len(params) < 3: raise ValueError(f"Not enough parameters found in line: {params}")
        lens_name = params[1].strip("'\"")
        lens_params = [float(x) for x in params[2:]]
        set_lens_dict[lens_name] = lens_params
    if set_lens_dict:
        for lens_name, params in set_lens_dict.items():
            set_lens_dict[lens_name] = params[1:]

    df_input = pd.DataFrame()
    rows_input = []
    max_param_len_input = 0
    for lens_name, params in set_lens_dict.items():
        row = {'Lens Name': lens_name}
        for i, val in enumerate(params): row[f'param{i+1}'] = val
        rows_input.append(row)
        if len(params) > max_param_len_input: max_param_len_input = len(params)
    columns_input = ['Lens Name'] + [f'param{i+1}' for i in range(max_param_len_input)]
    df_input = pd.DataFrame(rows_input, columns=columns_input)
    
    set_flag_lines = [line for line in py if line.startswith('glafic.setopt_lens(')]
    if not set_flag_lines: raise ValueError("No lines starting with 'glafic.setopt_lens(' found in the file.")
    set_flag_params = []
    for line in set_flag_lines:
        match = re.search(r'setopt_lens\((.*?)\)', line)
        if match:
            params_str = match.group(1)
            params = [param.strip() for param in params_str.split(',')]
            set_flag_params.append(params)
        else: raise ValueError(f"No valid parameters found in line: {line.strip()}")
    
    set_flag_dict = {}
    for params in set_flag_params:
        if len(params) < 2: raise ValueError(f"Not enough parameters found in line: {params}")
        lens_index = params[0].strip("'\"")
        lens_name = None
        for lens_params in set_lens_params:
            if lens_params[0].strip("'\"") == lens_index:
                lens_name = lens_params[1].strip("'\"")
                break
        if lens_name is None: raise ValueError(f"Lens name for index {lens_index} not found")
        flag = ','.join(params[1:])
        set_flag_dict[lens_name] = flag
    if set_flag_dict:
        for lens_name, flag in set_flag_dict.items():
            flag_parts = flag.split(',')
            set_flag_dict[lens_name] = ','.join(flag_parts[1:])
    
    df_flag = pd.DataFrame()
    rows_flag = []
    max_flag_len = 0
    for flag in set_flag_dict.values():
        flag_parts = flag.split(',')
        if len(flag_parts) > max_flag_len: max_flag_len = len(flag_parts)
    for lens_name, flag in set_flag_dict.items():
        flag_parts = flag.split(',')
        row = {'Lens Name': lens_name}
        for i, val in enumerate(flag_parts): row[f'flag{i+1}'] = val
        rows_flag.append(row)
    columns_flag = ['Lens Name'] + [f'flag{i+1}' for i in range(max_flag_len)]  
    df_flag = pd.DataFrame(rows_flag, columns=columns_flag)
    
    dfs = []
    for i in range(num_lens_profiles):
        lens_name = df['Lens Name'][i]
        model_type = None
        for m_model in model_list:
            if m_model.lower() == lens_name.lower():
                model_type = m_model
                break
        if model_type is None: continue
        symbols = model_params[model_type][:7]
        row_input = pd.DataFrame([df_input.iloc[i, 1:8].values], columns=symbols)
        row_output = pd.DataFrame([df.iloc[i, 1:8].values], columns=symbols)
        row_flags = pd.DataFrame([df_flag.iloc[i, 1:8].values], columns=symbols)
        lens_df = pd.concat([row_input.assign(Type='Input'), row_output.assign(Type='Output'), row_flags.assign(Type='Flag')], ignore_index=True)
        lens_df.insert(0, 'Lens Name', lens_name)
        cols = lens_df.columns.tolist()
        cols.insert(1, cols.pop(cols.index('Type')))
        lens_df = lens_df[cols]
        dfs.append(lens_df)
    
    columnn_names = ['x', 'y', 'mag', 'pos_err', 'mag_err', '1', '2', '3']
    obs_point = pd.read_csv(obs_point_file, sep='\s+', header=None, skiprows=1, names=columnn_names)
    
    out_point_file = os.path.join(model_path, f'{model_ver}_point.dat')
    out_point = pd.read_csv(out_point_file, sep='\s+', header=None, skiprows=1, names=columnn_names)

    out_point.drop(columns=['mag_err', '1', '2', '3'], inplace=True)
    mask = abs(out_point['mag']) >= 1
    out_point = out_point[mask[:len(out_point)]].reset_index(drop=True)
    out_point['x_diff'] = abs(out_point['x'] - obs_point['x'])
    out_point['y_diff'] = abs(out_point['y'] - obs_point['y'])
    out_point['mag_diff'] = abs(abs(out_point['mag']) - abs(obs_point['mag']))
    out_point['pos_sq'] = np.sqrt((out_point['x_diff']**2 + out_point['y_diff']**2).astype(float))
    pos_rms = np.average(out_point['pos_sq'])
    mag_rms = np.average(np.sqrt((out_point['mag_diff']**2).astype(float)))
    return pos_rms, mag_rms, dfs, chi2_value, source_params

In [9]:
pos_rms, mag_rms, dfs, chi2, source = rms_extract(model_name, model_output_dir, temp_input_py_file)
if pos_rms == -1: raise IOError("rms_extract failed.")

num_images = 0
out_point_file = os.path.join(model_output_dir, f'{model_name}_point.dat')
if os.path.exists(out_point_file):
    with open(out_point_file, 'r') as f: num_images = sum(1 for line in f if line.strip()) - 1
    
result_dict = {
    'm': m_val, 'n': n_val, 'o': o_val, 
    'num_images': num_images, 'pos_rms': pos_rms, 
    'mag_rms': mag_rms, 'chi2': chi2
}

num_lenses = model_name.count('_') - 2
macro_model_params = model_name.strip().split('_')[0]
if macro_model_params == 'NFW': 
    macro_model_params = 'ANFW'
macro_columns = model_params[macro_model_params]

if num_lenses > 1:
    for i in range(1, num_lenses):
        micro_model_params = model_name.strip().split('_')[i]
        if micro_model_params == 'SHEAR': micro_model_params = 'PERT'
        micro_columns = model_params[micro_model_params]
        micro_columns = [f'{col}_{i}' for col in micro_columns]

macro_cols = list(dict.fromkeys(macro_columns[:7])) if isinstance(macro_columns, list) else [macro_columns]
if num_lenses > 1:
    micro_cols = list(dict.fromkeys(micro_columns[:7])) if isinstance(micro_columns, list) else [micro_columns]

def _safe_get(dfs_idx, col):
    if len(dfs) > dfs_idx and col in dfs[dfs_idx].columns:
        return dfs[dfs_idx][col].iloc[1]
    return 0

for col in macro_cols: result_dict[col] = _safe_get(0, col)
if num_lenses > 1:
    for len_num in range(1, num_lenses):
        for col in micro_cols:
            stripped_col = col.strip(f'_{len_num}')
            result_dict[col] = _safe_get(1, stripped_col)

result_dict['source_x'] = source[0][1] if source and len(source) > 0 else 0
result_dict['source_y'] = source[0][2] if source and len(source) > 0 else 0

In [10]:
result_dict

{'m': 0.5,
 'n': 0.3,
 'o': 0.7,
 'num_images': 5,
 'pos_rms': 0.5206431707220018,
 'mag_rms': 18.6125,
 'chi2': 108000.1,
 'M': 1877439000000.0,
 'x': 20.86964,
 'y': 20.94406,
 'e': 0.107,
 '$θ_{e}$': 23.38,
 'c or $r_{s}$': 20.0,
 'NaN': 0.0,
 'source_x': 20.86232,
 'source_y': 20.94153}